# A deep state-space model for a consumer credit risk portfolio

This notebook outlines the development of a deep state-space model for consumer credit risk, built using [pyro.ai](https://pyro.ai/). At its core, the model employs Monte Carlo simulations for each loan, progressing through monthly timesteps. The hidden state at each step represents the loan’s status, with all accounts initially starting as current. From there, loans may transition to early payoff, arrears, or more commonly, remain current and advance to the next month.

The model requires a single primary input: a vector of anticipated cashflows, representing the loan’s installment schedule. The output used for training is the corresponding sequence of realized cashflows, i.e., the actual payments made. Behind the scenes, the model also trains an embedding based on the loan account identifier, which effectively captures the performance characteristics of each specific loan. This embedding serves several purposes, including:
- Simulating the performance of the existing portfolio.
- Extending the installment schedule to maturity to estimate the portfolio’s value if allowed to run off.
- Providing a low-dimensional representation of loan performance, enabling broader analysis beyond traditional good/bad account classifications for training applicant-level models.
- Reducing to a single risk dimension that represents the probability of default over any given time horizon.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pylab as plt

In [3]:
import torch
import torch.nn.functional as F
import pyro

In [4]:
from monteloanco import Model, Guide, GroupedBatchSampler, tmat_reshape, Template

We take a subset of the 2+ million accounts available here for speed.

In [5]:
df_train = pd.read_json('training.jsonl.gz', lines=True)
pd.testing.assert_index_equal(df_train.index, pd.RangeIndex(0, len(df_train)))

In [6]:
df_train.pymnt = df_train.pymnt.apply(torch.tensor)

The model has been designed such that it can train / simulate a large number of accounts in parallel on a GPU. If you don't have a suitable GPU installed on your machine simply replace `cuda:0` here with `cpu`. To achieve this we need to consider how the data is to be fed into the model. One of consideration with batching the tasks is that it is preferable to present the longest sequences first as these contain the most information, but more importanly than that, that all sequences in a batch ultimately have the same length. We pad the sequences out with 0, which is perfectly applicable to both the expected and realised payment seqences.

In [7]:
device = 'cuda:0'

In [8]:
batch_size = 100_000
dataset = df_train[['id', 'installment', 'loan_amnt', 'int_rate', 'pymnt']].to_dict(orient='records')
grouped_batch_sampler = GroupedBatchSampler(dataset, batch_size)

With the dataset batches defined it's time to run the optimisation process, and tune the parameters. The loss here is the difference between the anticipated payment and that that was made, for every account, up to and including each timestep in the sequence.

In [9]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [18]:
%%time

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# Initialize the model and guide
embedding_size=3
model = Model(embedding_size, device).to(device)
guide = Guide(embedding_size, device).to(device)

# Set up the optimizer and inference algorithm
optimizer = pyro.optim.Adam({"lr": 0.01})
svi = pyro.infer.SVI(model=model, guide=guide, optim=optimizer, loss=pyro.infer.Trace_ELBO())

# Run inference
#num_batches = grouped_batch_sampler.__len__()
num_iterations = 5_000
with tqdm(total=num_iterations, desc="Epochs", position=0) as epoch_pbar:
    for step in range(num_iterations):
    #with tqdm(total=num_batches, desc=f"Epoch {step + 1}", position=1, leave=False) as batch_pbar:
        losses = []
        for batch_id, batch in enumerate(DataLoader(dataset, batch_sampler=grouped_batch_sampler, num_workers=1)):
            losses.append(svi.step(
                batch_id=batch_id,
                batch_idx=torch.arange(len(batch['id'])).to(device), 
                installments=batch['installment'].to(device), 
                loan_amnt=batch['loan_amnt'].to(device), 
                int_rate=batch['int_rate'].to(device),
                pymnts=batch['pymnt'].to(device)))
            #batch_pbar.update(1)
        if step % np.ceil(num_iterations/100) == 0:
            print(f"Step {step} : Loss = {np.sum(losses)}")
        epoch_pbar.update(1)


Epochs:   0%|          | 0/5000 [00:00<?, ?it/s]

Step 0 : Loss = 179513174.94888005
Step 50 : Loss = 152808030.31364748
Step 100 : Loss = 136507471.32227212
Step 150 : Loss = 123957892.14333612
Step 200 : Loss = 114616215.49660581
Step 250 : Loss = 108133709.94140951
Step 300 : Loss = 101994388.39356217
Step 350 : Loss = 94452427.7038106
Step 400 : Loss = 87016696.77804813
Step 450 : Loss = 82656989.01834819
Step 500 : Loss = 77728096.65529737
Step 550 : Loss = 71695987.24740984
Step 600 : Loss = 68852649.40374424
Step 650 : Loss = 64373449.72738678
Step 700 : Loss = 60715532.34186183
Step 750 : Loss = 56914362.480946854
Step 800 : Loss = 53851087.561536625
Step 850 : Loss = 52745138.856184855
Step 900 : Loss = 51079762.92549653
Step 950 : Loss = 48219320.037238196
Step 1000 : Loss = 48017657.90611365
Step 1050 : Loss = 45616838.885054015
Step 1100 : Loss = 44170515.07818867
Step 1150 : Loss = 44043219.4031905
Step 1200 : Loss = 42879993.89277851
Step 1250 : Loss = 41977643.44021333
Step 1300 : Loss = 41393454.89681393
Step 1350 : Lo

Save model parameters to a file for inference in another notebook.

In [19]:
model.to(device)

Model()

In [21]:
pyro.get_param_store().save('param_store.pt')
torch.save(model.state_dict(), 'model_params.pt')
torch.save(guide.state_dict(), "guide_params.pt")